In [1]:
!pip install dask[complete] modin[all] ray pandas pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing insta

In [2]:
# Download ~2.5 GB of data from NYC Taxi data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# Convert to CSV to simulate a raw format
import pandas as pd
df = pd.read_parquet("yellow_tripdata_2023-01.parquet")
df.to_csv("yellow_tripdata.csv", index=False)


--2025-06-12 11:44:55--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.50, 65.8.245.171, 65.8.245.51, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  80.6MB/s    in 0.6s    

2025-06-12 11:44:56 (80.6 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [11]:
import pandas as pd
import time

start = time.time()
df_pandas = pd.read_csv("yellow_tripdata.csv")
end = time.time()
print("Pandas read time:", end - start)


Pandas read time: 9.657840251922607


In [12]:
import dask.dataframe as dd

start = time.time()
df_dask = dd.read_csv("yellow_tripdata.csv")
df_dask.head()  # Trigger computation
end = time.time()
print("Dask read time:", end - start)


Dask read time: 2.04052472114563


In [15]:
import ray
import modin.pandas as mpd

ray.init()
start = time.time()
df_modin = mpd.read_csv("yellow_tripdata.csv")
end = time.time()
print("Modin read time:", end - start)


2025-06-12 11:51:00,050	INFO worker.py:1917 -- Started a local Ray instance.


ValueError: An application is trying to access a Ray object whose owner is unknown(00ffffffffffffffffffffffffffffffffffffff0100000003e1f505). Please make sure that all Ray objects you are trying to access are part of the current Ray session. Note that object IDs generated randomly (ObjectID.from_random()) or out-of-band (ObjectID.from_binary(...)) cannot be passed as a task argument because Ray does not know which task created them. If this was not how your object ID was generated, please file an issue at https://github.com/ray-project/ray/issues/

In [14]:
ray.shutdown()

In [16]:


@ray.remote
def load_file():
    import pandas as pd
    return pd.read_csv("yellow_tripdata.csv")

start = time.time()
df_ray = ray.get(load_file.remote())
end = time.time()
print("Ray read time:", end - start)


(load_file pid=2493) <ipython-input-16-1536582613>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


Ray read time: 13.204901933670044


In [17]:
def clean_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)
    )
    return df

df_clean = clean_columns(df_pandas)


In [18]:
import yaml

schema = {
    'separator': '|',
    'columns': list(df_clean.columns)
}

with open("schema.yaml", "w") as f:
    yaml.dump(schema, f)


In [19]:
with open("schema.yaml") as f:
    yaml_schema = yaml.safe_load(f)

assert list(df_clean.columns) == yaml_schema['columns'], "Column names do not match!"
print("Validation successful!")


Validation successful!


In [20]:
df_clean.to_csv("cleaned_data.txt.gz", sep="|", index=False, compression="gzip")


In [21]:
import os
import gzip

rows, cols = df_clean.shape
file_size = os.path.getsize("cleaned_data.txt.gz") / (1024 * 1024)  # in MB

print(f"Total Rows: {rows}")
print(f"Total Columns: {cols}")
print(f"Compressed File Size: {file_size:.2f} MB")


Total Rows: 3066766
Total Columns: 19
Compressed File Size: 53.37 MB
